In [1]:
!pip install faiss-cpu

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [3]:
import json
import requests
def get_embedding(text, model="text-embedding-ada-002"):
        url = "https://api.openai.com/v1/embeddings"
        headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer sk-xxxxx"
        }
        data = {
            "input": text,
            "model": model
        }
        response = requests.post(url, headers=headers, data=json.dumps(data))
        response_data = response.json()
        embedding = response_data['data'][0]['embedding']
        return embedding

In [10]:
import openai
import faiss
import numpy as np

# 设置 OpenAI API 密钥
openai.api_key = 'sk-xxxx'

embedding_dim = 1536
index = faiss.IndexFlatL2(embedding_dim)


# 假设您有一系列问题和对应的答案
questions = ["问题1", "问题2", "问题3"]
answers = ["答案1", "答案2", "答案3"]

# 获取问题和答案的嵌入向量，并将其添加到索引中
for question, answer in zip(questions, answers):
    prompt = f"问题：{question}\n答案：{answer}\n"
    embedding = get_embedding(prompt)
    print(len(embedding))
    embedding = np.array([embedding]).astype('float32')  # 将嵌入向量转换为二维数组
    print(embedding)
    index.add(embedding)



1536
[[ 0.00981758 -0.01562339  0.00726885 ... -0.00707687 -0.02936668
  -0.02152851]]
1536
[[-0.00231937 -0.0160774   0.00879795 ... -0.01272739 -0.04225146
  -0.01471874]]
1536
[[ 0.00199679 -0.01804479  0.01160396 ... -0.00750883 -0.02725719
  -0.02796483]]


In [11]:
# 假设您有一个新的查询问题
query = "我想了解问题2的答案"

# 获取查询问题的嵌入向量
query_embedding = get_embedding(query)
query_embedding = np.array([query_embedding]).astype('float32')

# 使用 Faiss 索引进行相似度匹配和检索
k = 1  # 返回与查询问题最相似的问题-答案对的数量
distances, indices = index.search(query_embedding, k)

# 根据索引获取相似的问题-答案对
similar_question = questions[indices[0][0]]
similar_answer = answers[indices[0][0]]

print(f"查询问题：{query}")
print(f"最相似的问题：{similar_question}")
print(f"对应的答案：{similar_answer}")


查询问题：我想了解问题2的答案
最相似的问题：问题2
对应的答案：答案2


In [ ]:
import csv
import faiss
import numpy as np

import openai

# 设置 OpenAI API 密钥
openai.api_key = 'sk-xxxx'

embedding_dim = 1536
index = faiss.IndexFlatL2(embedding_dim)

# 定义函数获取嵌入向量
def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model)['data'][0]['embedding']

# 从 CSV 文件中读取问题和答案
csv_file = "qa_data.csv"

questions = []
answers = []

with open(csv_file, "r") as f:
    reader = csv.reader(f)
    next(reader)  # 跳过标题行
    for row in reader:
        question = row[0]
        answer = row[1]
        # print(question,answer)
        questions.append(question)
        answers.append(answer)

# 获取问题和答案的嵌入向量，并将其添加到索引中
for question, answer in zip(questions, answers):
    prompt = f"问题：{question}\n答案：{answer}\n"
    embedding = get_embedding(prompt)
    embedding = np.array([embedding]).astype('float32')  # 将嵌入向量转换为二维数组
    index.add(embedding)

# 保存 Faiss 索引到本地文件
index_file = "faiss_index.index"

faiss.write_index(index, index_file)


In [5]:
import faiss
import numpy as np
import csv

import openai

# 设置 OpenAI API 密钥
openai.api_key = 'sk-xxxxx'

embedding_dim = 1536

# 加载 Faiss 索引
index_file = "faiss_index.index"

index = faiss.read_index(index_file)

# 加载 QA 数据
qa_data_file = "qa_data.csv"


# 定义函数根据当前问题查找答案
def search_answer(question, index, questions, answers):
    question_embedding = get_embedding(question)
    question_embedding = np.array([question_embedding]).astype('float32')  # 将查询向量转换为二维数组

    # 在 Faiss 索引中搜索最相似的嵌入向量
    top_k = 1  # 返回前1个最相似的结果
    _, indices = index.search(question_embedding, top_k)

    # 根据索引获取相似的答案
    result = []
    for idx in indices[0]:
        question = questions[idx]
        answer = answers[idx]
        result.append((question, answer))

    return result

# 读取 QA 数据并存储问题和答案
questions = []
answers = []

with open(qa_data_file, "r") as f:
    reader = csv.reader(f)
    next(reader)  # 跳过标题行
    for row in reader:
        question = row[0]
        answer = row[1]
        questions.append(question)
        answers.append(answer)

# 示例使用
question = "今天光伏发电量是123 风电 154 制作一份csv，包含"
results = search_answer(question, index, questions, answers)

# 打印答案
for question, answer in results:
    print("Question:", question)
    print("Answer:", answer)
    print()


Question: 今天光伏发电量是123 风电 154 制作一份csv，包含这7天的发电预测数据和对应的厂用率给出均值和方差和预测的区间 文件名就叫今天日期～7天后的日期.csv
Answer: import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from get_predict import PredictFd
from get_cylpredict import PredictCyl

# 当前日期和7天后的日期
now_date = datetime.now().strftime('%Y%m%d')
future_date = (datetime.now() + timedelta(days=7)).strftime('%Y%m%d')

# 初始化数据列表
data = []

# 循环获取未来7天的预测数据
for i in range(8):
    predict_date = (datetime.now() + timedelta(days=i)).strftime('%Y%m%d')
    # 光伏发电量预测
    pv_predict = await PredictFd(now_date, predict_date, "123", "光伏")
    # 风电发电量预测
    wind_predict = await PredictFd(now_date, predict_date, "154", "风力")

    # 获取光伏的厂用率预测
    pv_cyl_predict = await PredictCyl(pv_predict['predict'], "gf")
    # 获取风电的厂用率预测
    wind_cyl_predict = await PredictCyl(wind_predict['predict'], "fd")

    # 添加到数据列表
    data.append({
        "date": predict_date,
        "pv_predict": pv_predict['predict'],
        "wind_predict": wind_predict